In [2]:
#
# BlackJack

import random

values = {
    'Two':2, 'Three':3, 'Four':4, 'Five':5,
    'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 
    'Jack':10, 'Queen':10, 'King':10, 'Ace':11
}

class Card():
    '''
    Definition of Card
    '''
    def __init__(self, suit, name):
        self.suit = suit
        self.name = name
    
    # print() support
    def __str__(self):
        return f'{self.name} of {self.suit}'

class Deck():
    '''
    Definition of Deck 
    '''
    
    suits = ('Hearts', 'Spades', 'Diamonds', 'Clubs')
    ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
    
    deck = []
    
    def __init__(self):
        for suit in Deck.suits:
            for rank in Deck.ranks:
                self.deck.append(Card(suit, rank))
    
    def shuffle(self):
        random.shuffle(Deck.deck)
        
    def deal(self):
        return Deck.deck.pop()
    
    # print() support - debugging purposes only.
    def __str__(self):
        print('-_-_- Debugging deck: -_-_-')
        deckStr = ''
        for card in Deck.deck:
            deckStr += card.__string__()
        return deckStr
    
class Hand():
    '''
    Definition of a player's hand.
    '''
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0
    
    def add_card(self, newCard):
        self.value += values[newCard.name]
        self.cards.append(newCard)
        
        if newCard.name == 'Ace':
            self.aces += 1
    
    def adjust_ace_value(self):
        while self.value > 21 and self.aces >= 1:
            self.aces -= 1
            self.value -= 10
    
class Chips():
    '''
    Keep track of starting chips, bets, and ongoing winnings.
    '''
    
    def __init__(self):
        self.total = 100
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lost_bet(self):
        self.total -= self.bet
    
    def pushed(self):
        self.bet = 0
    
    def place_bet(self):
        while True:
            try:
                betValue = int(input('Place your bet: '))
            except:
                print('Value needs to be a number.')
                continue
            else:
                if 0 < betValue <= self.total:
                    self.bet = betValue
                    break
                elif betValue <= 0:
                    print('Please enter a valid bet.')
                    continue
                else:
                    print('You don\'t have that much.')
                    continue
    
    def __str__(self):
        return f'\nYou have {self.total} chips.'
    
### 
# Helper function definitions
#

def hit(deck, hand):
    newCard = deck.deal()
    hand.add_card(newCard)
    
def hit_or_stand(deck,hand):
    global playing  # to control an upcoming while loop
    while True:
        try:
            response = input('Would you like to hit or stay').upper()
        except:
            print('Try again.')
            continue
        if response == 'H' or response == 'HIT':
            hit(deck, hand)
            break
        elif response == 'S' or response == 'STAY':
            playing = False
            break
            
def show_some(player,dealer):    
    print(f'\n-=-=- Player\'s hand: {player.value} -=-=-')
    for card in player.cards:
        print(card)
    
    print('\n-=-=- Dealer\'s hand -=-=-')
    for idx, card in enumerate(dealer.cards):
        if idx == 0:
            print('**hidden**')
        else:
            print(card)
    
def show_all(player,dealer):    
    print(f'\n-=-=- Player\'s hand: {player.value} -=-=-')
    for card in player.cards:
        print(card)
        
    print(f'\n-=-=- Dealer\'s hand {dealer.value} -=-=-')
    for card in dealer.cards:
        print(card)
        
def player_wins(chips):
    print('\nYou won!')
    chips.win_bet()
    pass
        
def player_loses(chips):
    print('\nYou Lost.')
    chips.lost_bet()
    pass

def play_again():
    print('*')
    while True:
        try:
            print('**')
            response = input("Would you like to play again? (Y/N)")
            print('NOT SEEING THIS.')
            response = response.upper()
        except:
            print('Try again.')
            continue
        else:
            return response == 'Y' or response == 'YES'
        finally:
            print('a')
    

In [3]:
# Set up the Player's chips
player_chips = Chips()

while True:
    # Print an opening statement
    print('Starting game of Black Jack.')
    
    # Prompt the Player for their bet
    print(player_chips)
    player_chips.place_bet()
    
    # Create & shuffle the deck
    deck = Deck()
    deck.shuffle()
    
    # deal two cards to each player
    player = Hand()
    dealer = Hand()
    
    hit(deck, player)
    hit(deck, dealer)
    hit(deck, player)
    hit(deck, dealer)
    
    # Show cards (but keep one dealer card hidden)
    show_some(player, dealer)
    
    if player.value == 21 and dealer.value == 21:
        player_chips.pushed()
        playing=False
    elif player.value == 21: 
        playing = False
        player_wins(player_chips)
    elif dealer.value == 21:
        playing = False
        player_loses(player_chips)
    else:
        playing = True
        
        
    while playing:  # recall this variable from our hit_or_stand function
        
        # Prompt for Player to Hit or Stand
        hit_or_stand(deck, player)
        
        # account for Aces
        # TODO: move to hit()
        player.adjust_ace_value()
        
        # Show cards (but keep one dealer card hidden)
        show_some(player, dealer)        
        
        if player.value == 21:
            break
            
        # If player's hand exceeds 21          
        if player.value > 21:
            player_loses(player_chips)
            playing = False
            break

    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    while player.value <= 21:        
        # Show all cards
        show_all(player, dealer)
        
        # account for Aces
        dealer.adjust_ace_value()
        
        # Run different winning scenarios
        if dealer.value > 21:
            player_wins(player_chips)
            break
        elif dealer.value > player.value:
            player_loses(player_chips)
            break
        elif dealer.value < 17:
            hit(deck, dealer)
        else:
            if dealer.value == player.value:
                print('Push!')
                player_chips.pushed()
                break
            elif dealer.value > player.value:
                player_loses(player_chips)
                break
            else:
                player_wins(player_chips)
                break
    
    # Inform Player of their chips total 
    print("\nPlayer's winnings stand at",player_chips.total)
    
    if player_chips.total > 0:
        # Ask to play again
        new_game = input("Would you like to play another hand? Enter 'y' or 'n' ")

        if new_game[0].lower()=='y':
            playing=True
            continue
        else:
            print("Thanks for playing!")
            print(f'Enjoy your {player_chips.total} chips!')
            break
    else:
        print("Better Luck Next Time!")
        break 

Starting game of Black Jack.

You have 100 chips.
Place your bet: 10

-=-=- Player's hand: 12 -=-=-
Six of Diamonds
Six of Clubs

-=-=- Dealer's hand -=-=-
**hidden**
Nine of Hearts
Would you like to hit or stayh

-=-=- Player's hand: 14 -=-=-
Six of Diamonds
Six of Clubs
Two of Spades

-=-=- Dealer's hand -=-=-
**hidden**
Nine of Hearts
Would you like to hit or stayh

-=-=- Player's hand: 24 -=-=-
Six of Diamonds
Six of Clubs
Two of Spades
Jack of Spades

-=-=- Dealer's hand -=-=-
**hidden**
Nine of Hearts

You Lost.

Player's winnings stand at 90
Would you like to play another hand? Enter 'y' or 'n' n
Thanks for playing!
Enjoy your 90 chips!
